In [23]:
import pandas as pd
import gspread
from pymisp import PyMISP, MISPAttribute
import plotly.graph_objects as go
import warnings

1. Create a PowerAutomate Workflow From *'Save a Tweets to a Google Sheet'* Template to Scrap/Extract Tweets with #Phishing
2. Configure a Google Cloud service to interact with the Google Sheet.

In [2]:
gc = gspread.service_account(filename=r".\phishingioc-keys.json")
sh = gc.open("PhishingIoCs")
worksheet = sh.sheet1

In [3]:
# Regex patterns
patternIP4 = r"(?P<ip4>[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3})"
patternUrl = r"(?P<url>hxxps?://[^\s]+)"
patternEma = r"(?P<email>[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7})"
patternDom = r"(?P<domain>(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\[\.\])+[a-z0-9][a-z0-9-]{0,61}[a-z0-9])"

In [4]:
# Tweet Text/Body
IoCRaw_list = worksheet.col_values(3)
IoC_raw = pd.DataFrame(IoCRaw_list[1:], columns = ['TweetBody'])

In [5]:
# Tweet Id
Tid_list = worksheet.col_values(7)
IoC_df = pd.DataFrame(Tid_list[1:], columns = ['TweetID'])

In [6]:
# Extract IoCs
IoC_df['IoCURLs'] = IoC_raw.TweetBody.str.extract(patternUrl)
IoC_df['IoCIPv4s'] = IoC_raw.TweetBody.str.extract(patternIP4)
IoC_df['IoCEmails'] = IoC_raw.TweetBody.str.extract(patternEma)
IoC_df['IoCDomains'] = IoC_raw.TweetBody.str.extract(patternDom)

In [7]:
IoC_df

,TweetID,IoCURLs,IoCIPv4s,IoCEmails,IoCDomains
0,1629050806971297792,hxxp://rogil75702[.]temp[.]swtest[.]ru/c49f9,NaN,NaN,rogil75702[.]temp[.]swtest[.]ru
1,1629058798240337924,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site
2,1629058762970365954,hxxps://paypay-login[.]huahuayufeng[.]com/all/...,155.94.128.125,NaN,paypay-login[.]huahuayufeng[.]com
3,1629058784483020800,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site
4,1629072062672408576,hxxps://metamasek.cc/,103.149.92.146,NaN,NaN
...,...,...,...,...,...
114,1630807958715023360,NaN,NaN,NaN,NaN
115,1630823171052937217,NaN,NaN,NaN,NaN
116,1630839290623062016,NaN,NaN,NaN,NaN
117,1630854386594500608,NaN,NaN,NaN,NaN


In [8]:
# Drop NaN Values on IoCs Columns
IoC_df_NNAN = IoC_df.dropna(subset=['IoCURLs', 'IoCIPv4s', 'IoCEmails', 'IoCDomains'], how='all').reset_index(drop=True)
IoC_df_NNAN

,TweetID,IoCURLs,IoCIPv4s,IoCEmails,IoCDomains
0,1629050806971297792,hxxp://rogil75702[.]temp[.]swtest[.]ru/c49f9,NaN,NaN,rogil75702[.]temp[.]swtest[.]ru
1,1629058798240337924,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site
2,1629058762970365954,hxxps://paypay-login[.]huahuayufeng[.]com/all/...,155.94.128.125,NaN,paypay-login[.]huahuayufeng[.]com
3,1629058784483020800,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site
4,1629072062672408576,hxxps://metamasek.cc/,103.149.92.146,NaN,NaN
5,1629086571063349249,hxxps://demo3[.]cloudwp[.]dev/trial-2w15v757/r...,NaN,NaN,demo3[.]cloudwp[.]dev
6,1629147729430552578,NaN,186.2.171.2,NaN,buradanaksen[.]site
7,1629161485929783298,hxxp://xn--80aaahorunoimsf3p.xn--p1ai/apps/,NaN,NaN,NaN
8,1629191709438492672,hxxps://accessolocalizzato[.]com/,NaN,NaN,accessolocalizzato[.]com
9,1629222421105344512,NaN,NaN,NaN,myshopify[.]com


In [9]:
# Types Of IoCs by Bar Chart
x = ["Types Of Indicators"]
Sercounts = IoC_df_NNAN[['IoCURLs', 'IoCIPv4s', 'IoCEmails', 'IoCDomains']].apply(lambda x: sum(x.notnull()))

fig = go.Figure(go.Bar(x=x, y=[Sercounts.IoCURLs], name='URLs'))
fig.add_trace(go.Bar(x=x, y=[Sercounts.IoCIPv4s], name='IPv4s'))
fig.add_trace(go.Bar(x=x, y=[Sercounts.IoCEmails], name='Emails'))
fig.add_trace(go.Bar(x=x, y=[Sercounts.IoCDomains], name='Domains'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

3. Create an *Authentification Key* bound to the Publisher user. 

In [27]:
# Add Attribute From a given list to an Event
def add_attribute_fromList(MISPInstance, eventID, List, type):
    for attr in List:
        misp_attribute = MISPAttribute()

        misp_attribute.value = str(attr)
        misp_attribute.category = str("Network activity")
        misp_attribute.type = str(type)
        misp_attribute.add_tag("Phishing")

        MISPInstance.add_attribute(eventID, misp_attribute)

In [28]:
warnings.filterwarnings("ignore")

# Add Collected IoCs to Local MISP Instance
## Ignoring the certificate verification

misp_url = "https://localhost"
misp_key = "API-KEY"
misp_verifycert = False

myMISPInstance = PyMISP( misp_url , misp_key , misp_verifycert , "json" )

listOfUrls = list(IoC_df_NNAN['IoCURLs'].dropna())
listOfIpv4 = list(IoC_df_NNAN['IoCIPv4s'].dropna())
listOfEmails = list(IoC_df_NNAN['IoCEmails'].dropna())
listOfDomains = list(IoC_df_NNAN['IoCDomains'].dropna())

# Example
add_attribute_fromList(myMISPInstance, "3", listOfEmails, "email")
